<a href="https://colab.research.google.com/github/akansh12/3D_computer_vision/blob/main/workshop/DGCNN_PartSeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install ipyvolume
!pip install torchmetrics

1.11.0+cu113
     |████████████████████████████████| 7.9 MB 8.6 MB/s 
     |████████████████████████████████| 3.5 MB 8.5 MB/s 
     |████████████████████████████████| 2.5 MB 6.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 7.9 MB/s 
     |████████████████████████████████| 3.4 MB 51.2 MB/s 
     |████████████████████████████████| 260 kB 56.9 MB/s 
     |████████████████████████████████| 271 kB 72.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 409 kB 7.8 MB/s 


Setup external visualization tools and import relevant packages

In [3]:
from google.colab import output
output.enable_custom_widget_manager()

In [4]:
import os.path as osp
import numpy as np
import ipyvolume as ipv
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.cm as cm
import matplotlib as mpl


import torch
import torch.nn.functional as F
from torch_scatter import scatter
from torchmetrics.functional import jaccard_index

import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MLP, DynamicEdgeConv

Get the dataset.

> Make sure the google drive is mounted

In [5]:
data_root = "/content/drive/MyDrive/datasets/"

In [6]:
path = osp.join(data_root, 'ShapeNet')
category = 'Airplane'  # Pass in `None` to train on all categories.
transform = T.Compose([
    T.RandomTranslate(0.01),
    T.RandomRotate(15, axis=0),
    T.RandomRotate(15, axis=1),
    T.RandomRotate(15, axis=2)
])
pre_transform = T.NormalizeScale()
train_dataset = ShapeNet(path, category, split='trainval', transform=transform,
                         pre_transform=pre_transform)
test_dataset = ShapeNet(path, category, split='test',
                        pre_transform=pre_transform)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True,
                          num_workers=6)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False,
                         num_workers=6)

Extracting /content/drive/MyDrive/datasets/ShapeNet/shapenetcore_partanno_segmentation_benchmark_v0_normal.zip
Processing...
Done!
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [7]:
meta_colors = [
        {"label": "airplane_body","id": 0,"color": [152,223,138]},
        {"label": "airplane_wings","id": 1,"color": [174,199,232]},
        {"label": "airplane_tail","id": 2,"color": [255,105,180]},
        {"label": "airplane_engine","id": 3,"color": [31,119,180]},
        {"label": "bag_handle","id": 4,"color": [255,187,120]},
        {"label": "bag_body","id": 5,"color": [188,189,34]},
        {"label": "cap_peak","id": 6,"color": [140,86,75]},
        {"label": "cap_panel","id": 7,"color": [255,152,150]},
        {"label": "car_roof","id": 8,"color": [214,39,40]},
        {"label": "car_hood","id": 9,"color": [197,176,213]},
        {"label": "car_wheels","id": 10,"color": [148,103,189]},
        {"label": "car_body","id": 11,"color": [196,156,148]},
        {"label": "chair_back","id": 12,"color": [23,190,207]},
        {"label": "chair_seat","id": 13,"color": [186,85,211]},
        {"label": "chair_leg","id": 14,"color": [247,182,210]},
        {"label": "chair_arm","id": 15,"color": [66,188,102]},
        {"label": "earphone_earphone","id": 16,"color": [219,219,141]},
        {"label": "earphone_headband","id": 17,"color": [140,57,197]},
        {"label": "earphone_microphone","id": 18,"color": [202,185,52]},
        {"label": "guitar_head","id": 19,"color": [213,92,176]},
        {"label": "guitar_neck","id": 20,"color": [200,54,131]},
        {"label": "guitar_body","id": 21,"color": [92,193,61]},
        {"label": "knife_blade","id": 22,"color": [78,71,183]},
        {"label": "knife_handle","id": 23,"color": [172,114,82]},
        {"label": "lamp_base","id": 24,"color": [255,127,14]},
        {"label": "lamp_shape","id": 25,"color": [91,163,138]},
        {"label": "lamp_ceiling","id": 26,"color": [153,98,156]},
        {"label": "lamp_tube","id": 27,"color": [140,153,101]},
        {"label": "laptop_keyboard","id": 28,"color": [158,218,229]},
        {"label": "laptop_screen","id": 29,"color": [100,125,154]},
        {"label": "motorbike_gastank","id": 30,"color": [178,127,135]},
        {"label": "motorbike_seat","id": 31,"color": [120,185,128]},
        {"label": "motorbike_wheel","id": 32,"color": [146,111,194]},
        {"label": "motorbike_handle","id": 33,"color": [44,160,44]},
        {"label": "motorbike_light","id": 34,"color": [112,128,144]},
        {"label": "motorbike_body","id": 35,"color": [96,207,209]},
        {"label": "mug_handle","id": 36,"color": [227,119,194]},
        {"label": "mug_body","id": 37,"color": [51,176,203]},
        {"label": "pistol_barrel","id": 38,"color": [94,106,211]},
        {"label": "pistol_handle","id": 39,"color": [82,84,163]},
        {"label": "pistol_trigger","id": 40,"color": [100,85,144]},
        {"label": "rocket_body","id": 41,"color": [255,127,80]},
        {"label": "rocket_fin","id": 42,"color": [0,100,0]},
        {"label": "rocket_nose","id": 43,"color": [173,255,47]},
        {"label": "skateboard_wheel","id": 44,"color": [64,224,208]},
        {"label": "skateboard_deck","id": 45,"color": [0,255,255]},
        {"label": "skateboard_bearing","id": 46,"color": [25,25,112]},
        {"label": "table_top","id": 47,"color": [178,76,76]},
        {"label": "table_leg","id": 48,"color": [255,0,255]},
        {"label": "table_drawer","id": 49,"color": [152,223,138]}
]

id2color = {}
for ix in meta_colors:
    id2color[ix["id"]] = ix["color"]

print(id2color)

{0: [152, 223, 138], 1: [174, 199, 232], 2: [255, 105, 180], 3: [31, 119, 180], 4: [255, 187, 120], 5: [188, 189, 34], 6: [140, 86, 75], 7: [255, 152, 150], 8: [214, 39, 40], 9: [197, 176, 213], 10: [148, 103, 189], 11: [196, 156, 148], 12: [23, 190, 207], 13: [186, 85, 211], 14: [247, 182, 210], 15: [66, 188, 102], 16: [219, 219, 141], 17: [140, 57, 197], 18: [202, 185, 52], 19: [213, 92, 176], 20: [200, 54, 131], 21: [92, 193, 61], 22: [78, 71, 183], 23: [172, 114, 82], 24: [255, 127, 14], 25: [91, 163, 138], 26: [153, 98, 156], 27: [140, 153, 101], 28: [158, 218, 229], 29: [100, 125, 154], 30: [178, 127, 135], 31: [120, 185, 128], 32: [146, 111, 194], 33: [44, 160, 44], 34: [112, 128, 144], 35: [96, 207, 209], 36: [227, 119, 194], 37: [51, 176, 203], 38: [94, 106, 211], 39: [82, 84, 163], 40: [100, 85, 144], 41: [255, 127, 80], 42: [0, 100, 0], 43: [173, 255, 47], 44: [64, 224, 208], 45: [0, 255, 255], 46: [25, 25, 112], 47: [178, 76, 76], 48: [255, 0, 255], 49: [152, 223, 138]}


## Data Explore

In [8]:
def visualize_pc(points, color=False, distance=False):
    norm = mpl.colors.Normalize(vmin=0, vmax=1.5)
    cmap = cm.hsv
    dist = np.sqrt(np.sum((points - points.mean())**2, 1))
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    cols = m.to_rgba(dist)

    ipv.figure()
    if isinstance(color, np.ndarray):
        ipv.scatter(points[:, 0], points[:, 1], points[:, 2], marker='sphere', color=color)
    elif distance==True:
        ipv.scatter(points[:, 0], points[:, 1], points[:, 2], marker='sphere', color=cols)
    else:
        ipv.scatter(points[:, 0], points[:, 1], points[:, 2], marker='sphere', color='blue')
    ipv.show()

In [9]:
label = train_dataset[0].y.data.numpy()
points = train_dataset[0].pos.data.numpy()

colors = np.ones((label.shape[0], 4))
colors[:, :3] = np.array([id2color[cx] for cx in label])/255.0


In [10]:
colors.shape

(2518, 4)

In [11]:
visualize_pc(points, colors)

# Net

In [12]:
class Net(torch.nn.Module):
    def __init__(self, out_channels, k=30, aggr='max'):
        super().__init__()

        self.conv1 = DynamicEdgeConv(MLP([2 * 6, 64, 64]), k, aggr)
        self.conv2 = DynamicEdgeConv(MLP([2 * 64, 64, 64]), k, aggr)
        self.conv3 = DynamicEdgeConv(MLP([2 * 64, 64, 64]), k, aggr)

        self.mlp = MLP([3 * 64, 1024, 256, 128, out_channels], dropout=0.5,
                       batch_norm=False)

    def forward(self, data):
        x, pos, batch = data.x, data.pos, data.batch
        x0 = torch.cat([x, pos], dim=-1)
        x1 = self.conv1(x0, batch)
        x2 = self.conv2(x1, batch)
        x3 = self.conv3(x2, batch)
        out = self.mlp(torch.cat([x1, x2, x3], dim=1))
        return F.log_softmax(out, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(train_dataset.num_classes, k=30).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.8)

In [ ]:
def train():
    model.train()

    total_loss = correct_nodes = total_nodes = 0
    for i, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct_nodes += out.argmax(dim=1).eq(data.y).sum().item()
        total_nodes += data.num_nodes

        if (i + 1) % 10 == 0:
            print(f'[{i+1}/{len(train_loader)}] Loss: {total_loss / 10:.4f} '
                  f'Train Acc: {correct_nodes / total_nodes:.4f}')
            total_loss = correct_nodes = total_nodes = 0

In [ ]:
@torch.no_grad()
def test(loader):
    model.eval()

    ious, categories = [], []
    y_map = torch.empty(loader.dataset.num_classes, device=device).long()
    for data in loader:
        data = data.to(device)
        outs = model(data)

        sizes = (data.ptr[1:] - data.ptr[:-1]).tolist()
        for out, y, category in zip(outs.split(sizes), data.y.split(sizes),
                                    data.category.tolist()):
            category = list(ShapeNet.seg_classes.keys())[category]
            part = ShapeNet.seg_classes[category]
            part = torch.tensor(part, device=device)

            y_map[part] = torch.arange(part.size(0), device=device)

            iou = jaccard_index(out[:, part].argmax(dim=-1), y_map[y],
                                num_classes=part.size(0), absent_score=1.0)
            ious.append(iou)

        categories.append(data.category)

    iou = torch.tensor(ious, device=device)
    category = torch.cat(categories, dim=0)

    mean_iou = scatter(iou, category, reduce='mean')  # Per-category IoU.
    return float(mean_iou.mean())  # Global IoU.

In [ ]:
for epoch in range(1, 31):
    train()
    iou = test(test_loader)
    print(f'Epoch: {epoch:02d}, Test IoU: {iou:.4f}')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[10/235] Loss: 0.4314 Train Acc: 0.8374
[20/235] Loss: 0.4237 Train Acc: 0.8438
[30/235] Loss: 0.3970 Train Acc: 0.8540
[40/235] Loss: 0.3767 Train Acc: 0.8679
[50/235] Loss: 0.3979 Train Acc: 0.8598
[60/235] Loss: 0.3990 Train Acc: 0.8667
[70/235] Loss: 0.4038 Train Acc: 0.8669
[80/235] Loss: 0.3575 Train Acc: 0.8749


KeyboardInterrupt: ignored

In [ ]:
torch.save(model, os.path.join(data_root, "trained_model_partseg_10epochs.pth"))